In [10]:
import numpy as np
from pymoo.core.problem import Problem
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.sampling.rnd import PermutationRandomSampling
from pymoo.operators.crossover.ox import OrderCrossover
from pymoo.operators.mutation.inversion import InversionMutation
from pymoo.termination.default import DefaultSingleObjectiveTermination
from pymoo.optimize import minimize

In [11]:
class BakerySchedulingProblem(Problem):
    def __init__(self, user_sequence, recipe_id_to_index, processing_times, changeover_times, batch_sizes, tact_times):
        """
        Initialize the bakery scheduling problem with user sequence and parameters.

        Parameters:
        - user_sequence: List of recipe indices (e.g., [1, 0, 3, 4, 6, 1, 3]).
        - recipe_id_to_index: Dictionary mapping recipe IDs to indices (e.g., {1: 0, 3: 1, ...}).
        - processing_times: Array of shape [n_machines, n_recipes] with processing times.
        - changeover_times: Array of shape [n_machines, n_recipes, n_recipes] with changeover times.
        - batch_sizes: Array of shape [n_recipes] with batch sizes per recipe.
        - tact_times: Array of shape [n_recipes] with tact times per recipe.
        """
        self.seq_length = len(user_sequence)
        self.user_sequence = np.array(user_sequence)
        self.recipe_id_to_index = recipe_id_to_index
        super().__init__(
            n_var=self.seq_length,      # Number of positions in the sequence
            n_obj=1,                    # Single objective: minimize makespan
            n_constr=0,                 # No constraints
            xl=0,                       # Minimum index
            xu=self.seq_length - 1,     # Maximum index
            type_var=int                # Integer variables for permutation
        )
        self.n_machines = processing_times.shape[0]
        self.processing_times = processing_times
        self.changeover_times = changeover_times
        self.batch_sizes = batch_sizes
        self.tact_times = tact_times

    def _evaluate(self, X, out, *args, **kwargs):
        """Evaluate the makespan for each permutation in the population."""
        makespans = np.array([self.calculate_makespan(perm) for perm in X])
        # print('X:', X)
        # print('makespans:', makespans)
        out["F"] = makespans

    def calculate_makespan(self, perm):
        """Calculate makespan with postponement for no-wait scheduling."""
        seq = self.user_sequence[perm]  # Permuted sequence of recipe indices
        # print('seq:', seq)

        start_times = np.zeros((self.seq_length, self.n_machines))
        end_times = np.zeros((self.seq_length, self.n_machines))

        for i in range(self.seq_length):
            recipe = seq[i]
            if i == 0:
                # First recipe starts at time 0 on the first machine
                start_times[i, 0] = 0
            else:
                # Calculate postponement to satisfy no-wait condition
                prev_recipe = seq[i - 1]
                postponement_candidates = []
                # Machine 0: Earliest start after previous recipe
                min_start_m0 = end_times[i - 1, 0] + self.changeover_times[0, prev_recipe, recipe]
                # Check other machines for no-wait requirement
                for m in range(1, self.n_machines):
                    # Time when machine m is ready after previous recipe
                    machine_ready = end_times[i - 1, m] + self.changeover_times[m, prev_recipe, recipe]
                    # Cumulative processing time from machine 0 to m-1
                    cumulative_proc_time = sum(self.processing_times[k, recipe] for k in range(m))
                    # Required start time on machine 0 to reach machine m on time
                    required_start = machine_ready - cumulative_proc_time
                    postponement_candidates.append(required_start)
                # Start time is the maximum of all constraints
                start_times[i, 0] = min_start_m0
                if postponement_candidates:
                    start_times[i, 0] = max(min_start_m0, max(postponement_candidates))

            # Compute start and end times for all machines with no-wait condition
            for m in range(self.n_machines):
                if m > 0:
                    # Start on machine m immediately after end on machine m-1
                    start_times[i, m] = end_times[i, m - 1]
                # End time = start + (batch_size - 1) * tact_time + processing_time
                end_times[i, m] = (start_times[i, m] +
                                   (self.batch_sizes[i] - 1) * self.tact_times[recipe] +
                                   self.processing_times[m, recipe])

        # Makespan is the end time of the last recipe on the last machine
        return end_times[-1, -1]

In [12]:
# Example setup
# Define recipe IDs (7 unique recipes)
recipe_ids = [1, 3, 4, 5, 8, 12, 14]
recipe_id_to_index = {rid: idx for idx, rid in enumerate(recipe_ids)}  # e.g., {1: 0, 3: 1, ...}

n_unique_recipes = len(recipe_ids)  # Number of unique recipes

# User's sequence with possible duplicates (8 positions)
user_sequence_ids = [3, 1, 5, 8, 14, 3, 5, 4]
user_sequence_idx = [recipe_id_to_index[rid] for rid in user_sequence_ids]  # [1, 0, 3, 4, 6, 1, 3]

# Problem parameters
n_machines = 5
n_recipes = len(user_sequence_ids)

# Sample data (randomized for demonstration)
np.random.seed(42)
processing_times = np.random.randint(5, 20, size=(n_machines, n_unique_recipes))
changeover_times = np.random.randint(1, 5, size=(n_machines, n_unique_recipes, n_unique_recipes))
for m in range(n_machines):
    for i in range(n_unique_recipes):
        changeover_times[m, i, i] = 0  # No changeover time for the same recipe
batch_sizes = np.random.randint(10, 20, size=n_recipes)
tact_times = np.random.uniform(0.5, 2.0, size=n_unique_recipes)

print("Recipe IDs:\n", recipe_ids) 
print("Recipe ID to Index Mapping:\n", recipe_id_to_index)

print("User Sequence IDs:\n", user_sequence_ids)
print("User Sequence Indices:\n", user_sequence_idx)

print("Processing Times:\n", processing_times)
# print("Changeover Times:\n", changeover_times)

print("Batch Sizes:\n", batch_sizes)
print("Tact Times:\n", tact_times)

Recipe IDs:
 [1, 3, 4, 5, 8, 12, 14]
Recipe ID to Index Mapping:
 {1: 0, 3: 1, 4: 2, 5: 3, 8: 4, 12: 5, 14: 6}
User Sequence IDs:
 [3, 1, 5, 8, 14, 3, 5, 4]
User Sequence Indices:
 [1, 0, 3, 4, 6, 1, 3, 2]
Processing Times:
 [[11  8 17 19 15 12 17]
 [ 9 11 14  7 11 15 15]
 [12  9  8 12 12  7 10]
 [ 9  6 12 16 18 10  6]
 [16  9  5 16 14 10 17]]
Batch Sizes:
 [16 11 12 10 14 10 17 10]
Tact Times:
 [0.90017152 1.96492243 1.11655552 0.5495761  1.01760687 1.45152702
 1.52105818]


In [13]:
for m in range(1, n_machines):
    print('machine:', m)
    for k in range(m):
        print('k:', k)
    cumulative_proc_time = sum(processing_times[k, 0] for k in range(m))

    print('cumulative_proc_time:', cumulative_proc_time)
    print('--'*20)


machine: 1
k: 0
cumulative_proc_time: 11
----------------------------------------
machine: 2
k: 0
k: 1
cumulative_proc_time: 20
----------------------------------------
machine: 3
k: 0
k: 1
k: 2
cumulative_proc_time: 32
----------------------------------------
machine: 4
k: 0
k: 1
k: 2
k: 3
cumulative_proc_time: 41
----------------------------------------


In [14]:
recipe_id_to_index[5]

3

In [15]:
changeover_times[0, 1, 2]

4

In [16]:
# Instantiate the problem
problem = BakerySchedulingProblem(
    user_sequence=user_sequence_idx,
    recipe_id_to_index=recipe_id_to_index,
    processing_times=processing_times,
    changeover_times=changeover_times,
    batch_sizes=batch_sizes,
    tact_times=tact_times
)

# Configure the genetic algorithm
algorithm = GA(
    pop_size=len(user_sequence_idx) * 10,  # Population size
    sampling=PermutationRandomSampling(),
    crossover=OrderCrossover(),
    mutation=InversionMutation(),
    eliminate_duplicates=True
)

# Run optimization
res = minimize(
    problem,
    algorithm,
    # termination=('n_gen', 100),
    termination=DefaultSingleObjectiveTermination(period=50, n_max_gen=10000),
    seed=1,
    verbose=True  # Set to True to see progress
)

# Extract and display results
best_perm = res.X
best_sequence_idx = [user_sequence_idx[i] for i in best_perm]
best_sequence_ids = [recipe_ids[idx] for idx in best_sequence_idx]

print("Original sequence of recipe IDs:", user_sequence_ids)
print("Best permutation of indices:", best_perm)
print("Best sequence of recipe IDs:", best_sequence_ids)
print("Minimized makespan:", res.F[0])

n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |       80 |  7.149677E+02 |  6.708034E+02
     2 |      160 |  6.955618E+02 |  6.679685E+02
     3 |      240 |  6.881694E+02 |  6.679685E+02
     4 |      320 |  6.842276E+02 |  6.627829E+02
     5 |      400 |  6.798641E+02 |  6.627829E+02
     6 |      480 |  6.776467E+02 |  6.617829E+02
     7 |      560 |  6.749573E+02 |  6.617829E+02
     8 |      640 |  6.734766E+02 |  6.617829E+02
     9 |      720 |  6.724172E+02 |  6.617829E+02
    10 |      800 |  6.712269E+02 |  6.617829E+02
    11 |      880 |  6.703324E+02 |  6.617829E+02
    12 |      960 |  6.698633E+02 |  6.617829E+02
    13 |     1040 |  6.694040E+02 |  6.617829E+02
    14 |     1120 |  6.688291E+02 |  6.617829E+02
    15 |     1200 |  6.681772E+02 |  6.617829E+02
    16 |     1280 |  6.678340E+02 |  6.617829E+02
    17 |     1360 |  6.677328E+02 |  6.617829E+02
    18 |     1440 |  6.676911E+02 |  6.617829E+02
    19 |     1520 |  6.675967E+02 |  6.617829E+02


In [17]:
perm = np.array([6, 2, 1, 0, 4, 3, 5])
print("perm", perm)
user_sequence = np.array(user_sequence_idx)
print("user_sequence", user_sequence)

perm [6 2 1 0 4 3 5]
user_sequence [1 0 3 4 6 1 3 2]


In [18]:
seq = user_sequence[perm]  # Permuted sequence of recipe indices

print("seq:", seq, type(seq))


seq: [3 3 0 1 6 4 1] <class 'numpy.ndarray'>


The line `seq = user_sequence[perm]` is using **NumPy array indexing** to reorder the elements of the `user_sequence` array based on the indices specified in the `perm` array.

### Explanation:
1. **`user_sequence`**:
    - This is a NumPy array: `array([1, 0, 3, 4, 6, 1, 3])`.
    - It represents the sequence of recipe indices provided by the user.

2. **`perm`**:
    - This is a NumPy array: `array([6, 2, 1, 0, 4, 3, 5])`.
    - It represents a permutation of indices (positions) into the `user_sequence` array.

3. **`user_sequence[perm]`**:
    - This reorders the elements of `user_sequence` according to the indices in `perm`.
    - For example:
      - `perm[0] = 6` → `user_sequence[6] = 3`
      - `perm[1] = 2` → `user_sequence[2] = 3`
      - `perm[2] = 1` → `user_sequence[1] = 0`
      - And so on...

4. **Result**:
    - The resulting `seq` is: `array([3, 3, 0, 1, 6, 4, 1])`.

### Purpose:
This operation creates a new sequence (`seq`) by reordering the original `user_sequence` according to the permutation defined in `perm`. This is often used in optimization problems to evaluate different orderings or arrangements of items.